In [2]:
import pandas
import numpy as np

In [3]:
in_df = pandas.read_csv('/Users/jakevogel/Science/TADPOLE/KNN_50_cleaned.csv',index_col=0)
ref_df = pandas.read_csv('/Users/jakevogel/Science/TADPOLE/KNN_50_cleaned_ref.csv',index_col=0)

In [4]:
tref.head()

NameError: name 'tref' is not defined

In [5]:
rid = 2

In [6]:
ref_df.columns

Index(['RID', 'PTID', 'VISCODE', 'SITE', 'D1', 'D2', 'COLPROT', 'ORIGPROT',
       'EXAMDATE', 'DX_bl', 'y_DX', 'y_ADAS11', 'y_ADAS13', 'ventr_ICV_ratio'],
      dtype='object')

In [111]:
def create_test_table(in_df, ref_df, rids, mode = 'train'):
    '''
    If mode is set to train:
    
    Given the output dataframes from the previous functions, and a list of RIDs, 
    this function will create a "to-predict" dataframe for each subject. For each 
    subject time point, this dataframe will several identical rows of all X 
    variables and y variables for that subject and timepoint, but with a field 
    called "delta_t" changing so as to reflect the distance (in months) between 
    this time point and every *SUBSEQUENT* (not previous) timepoint. In addition.
    the row will have the "y" variables for the timepoint that the delta_t is 
    referring to. So, in other words, each timepoint will have a row for every
    subsequent timepoint, complete with a variable noting the time-distance between
    the two timepoints, and the y of that timepoint
    
    This means that, for a given subject, the shape of the output dataframe will be
    (N_timepoints * (N_timepoints+1)/2).
    
    The function will then append all subject dataframes together and return this
    large dataframe. 
    
    If mode is set to test:
    
    Given the output dataframes from the previous functions, and a list of RIDs,
    this function will create a "testing" dataframe for each subject. For each 
    subject time point, the dataframe will have a single row containing all X and
    Y variables for that subject-specific timepoint, as well as new variable
    "months_to_final_timepoint" which represent the amount of time betweeb this
    timepoint and the subject's final timepoint. Note that the subject's final
    timepoint will be removed.
    
    in_df = *_cleaned.csv from the previous function, as a pandas DataFrame
    ref_df = *_cleaned_ref.csv from the previous function, as a pandas DataFrame
    rid = a list or array of RIDs
    '''
    
    if mode not in ['train','test']:
        raise IOError('mode must be set to "train" or "test"')
    
    print('Making %sing table'%mode)
    
    if mode == 'train':
        
        catch = []
        for rid in rids:
            # create subject-specific inputs
            gind = ref_df[ref_df.RID==rid].index
            x_tbl = in_df.loc[gind].sort_values('Month_bl',ascending=True)
            y_tbl = ref_df.loc[x_tbl.index]
            # create empty output dataframe for subject
            N_timepoints = len(x_tbl) - 1
            N_rows = round((N_timepoints * (N_timepoints+1)/2))
            ycols = [x for x in y_tbl.columns if 'y_' in x]
            fys = ['future_%s'%x for x in ycols]
            cols = ['RID'] + x_tbl.columns.tolist() + ycols + ['delta_t'] + fys
            final_output = pandas.DataFrame(index=range(N_rows), columns = cols)
            # populate dataframe using a convoluted while loop
            time_vec = x_tbl['Month'].values
            cntr = 0
            cumul = 0
            while cntr < N_timepoints:
                jnk = pandas.concat([x_tbl.loc[x_tbl.index[cntr]],
                                     y_tbl.loc[y_tbl.index[cntr]][['RID']+ycols]])
                row = jnk[cols[:-1]]
                iTime = row['Month']
                dTs = [x - iTime for x in time_vec[(cntr+1):]]
                for i in range(len(dTs)):
                    final_output.loc[final_output.index[cumul+i],:] = row
                    final_output.loc[final_output.index[cumul+i],'delta_t'] = dTs[i]
                    final_output.loc[final_output.index[cumul+i],fys
                                    ] = y_tbl.loc[y_tbl.index[cntr+(i+1)]][ycols].values
                    
                
                cumul += len(dTs)
                cntr += 1
            catch.append(final_output)

    elif mode == 'test': 
        catch = []

        for rid in rids:
            # create subject-specific inputs
            gind = ref_df[ref_df.RID==rid].index
            x_tbl = in_df.loc[gind].sort_values('Month_bl',ascending=True)
            y_tbl = ref_df.loc[x_tbl.index]
            ycols = [x for x in y_tbl.columns if 'y_' in x]
            cols = ['RID'] + x_tbl.columns.tolist() + ycols + ['months_to_final_timepoint']
            # create output dataframe
            outdf = pandas.concat([x_tbl,y_tbl[['RID']+ycols]],axis=1)
            outdf = outdf.reindex(columns=cols)
            # calculate time to final timepoint
            fTime = x_tbl.loc[x_tbl.index[-1],'Month']
            deltaT = [fTime - x for x in x_tbl['Month']]
            outdf.loc[:,'months_to_final_timepoint'] = deltaT
            # drop final timepoint
            outdf.drop(outdf.index[-1],axis=0,inplace=True)
            
            catch.append(outdf)
            
    # combine subject specific dfs in to a single df 
    big_final_df = pandas.concat(catch)

    return big_final_df
    
    
        
        

In [108]:
tst= create_test_table(in_df, ref_df, [2,3,5])

Making training table


In [110]:
tst

,RID,AGE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,PTMARRY,APOE4,CDRSB,MMSE,...,ST99TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST99TS_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST9SV_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,y_DX,y_ADAS11,y_ADAS13,delta_t,future_y_DX,future_y_ADAS11,future_y_ADAS13
0,2,0.0695753,-0.87488,0.00410519,-0.172768,-0.253396,-0.48648,-0.819258,-0.772763,0.356549,...,1.42111,-0.606512,3.28765,1,-0.0839499,0.0988391,0.229416,1,-0.0839499,0.184332
1,2,0.0695753,-0.87488,0.00410519,-0.172768,-0.253396,-0.48648,-0.819258,-0.772763,0.356549,...,1.42111,-0.606512,3.28765,1,-0.0839499,0.0988391,1.37649,1,0.0701234,0.212544
2,2,0.0695753,-0.87488,0.00410519,-0.172768,-0.253396,-0.48648,-0.819258,-0.772763,0.356549,...,1.42111,-0.606512,3.28765,1,-0.0839499,0.0988391,2.29416,1,0.301813,0.469022
3,2,0.0695753,-0.87488,0.00410519,-0.172768,-0.253396,-0.48648,-0.819258,-0.772763,0.356549,...,1.42111,-0.606512,3.28765,1,-0.0839499,0.0988391,2.52357,NaN,0.167998,0.350433
4,2,0.0695753,-0.87488,0.00410519,-0.172768,-0.253396,-0.48648,-0.819258,-0.772763,0.356549,...,1.42111,-0.606512,3.28765,1,-0.0839499,0.0988391,2.75299,1,0.0701234,0.298037
5,2,0.0695753,-0.87488,0.00410519,-0.172768,-0.253396,-0.48648,-0.819258,-0.772763,0.356549,...,1.42111,-0.606512,3.28765,1,-0.0839499,0.0988391,2.9824,NaN,-0.0834568,0.0294039
6,2,0.0695753,-0.87488,0.00410519,-0.172768,-0.253396,-0.48648,-0.819258,-0.772763,0.356549,...,1.42111,-0.606512,3.28765,1,-0.0839499,0.0988391,3.21182,2,-0.27741,-0.300412
7,2,0.0695753,-0.87488,0.00410519,-0.172768,-0.253396,-0.48648,-0.819258,-0.772763,0.356549,...,1.42111,-0.606512,3.28765,1,-0.0839499,0.0988391,3.44123,NaN,-0.190362,-0.0872098
8,2,0.0695753,-0.87488,0.00410519,-0.172768,-0.253396,-0.48648,-0.819258,-0.772763,0.356549,...,1.42111,-0.606512,3.28765,1,-0.0839499,0.0988391,3.67065,1,-0.161566,0.041559
9,2,0.0695753,-0.87488,0.00410519,-0.172768,-0.253396,-0.48648,-0.819258,-0.772763,0.356549,...,1.42111,-0.606512,3.28765,1,-0.0839499,0.0988391,3.90007,NaN,-0.146173,0.0330878


In [81]:
rid = 2
gind = ref_df[ref_df.RID==rid].index
x_tbl = in_df.loc[gind].sort_values('Month_bl',ascending=True)
y_tbl = ref_df.loc[x_tbl.index]
ycols = [x for x in y_tbl.columns if 'y_' in x]
# create output dataframe
outdf = pandas.concat([x_tbl,y_tbl[['RID']+ycols]],axis=1)
# calculate time to final timepoint
fTime = x_tbl.loc[x_tbl.index,'Month']

In [82]:
fTime

0       -0.995944
5723    -0.766528
5724     0.380550
5725     1.298212
5726     1.527628
5727     1.757043
5728     1.986459
5729     2.215874
5730     2.445290
5731     2.674706
10836    2.904121
11571    3.133537
11959    3.592368
Name: Month, dtype: float64

In [94]:
tst2 = create_test_table(in_df, ref_df, [2,3,5],mode='test')

Making testing table


In [95]:
tst2

,RID,AGE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,PTMARRY,APOE4,CDRSB,MMSE,...,ST99CV_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST99SA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST99TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST99TS_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST9SV_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,y_DX,y_ADAS11,y_ADAS13,delta_t,months_to_final_timepoint
0,2,0.069575,-0.87488,0.004105,-0.172768,-0.253396,-0.48648,-0.819258,-0.772763,0.356549,...,3.176360,2.627168,1.421115,-0.606512,3.287650,1.0,-0.083950,0.098839,NaN,4.588312
5723,2,0.069575,-0.87488,0.004105,-0.172768,-0.253396,-0.48648,-0.819258,-0.772763,0.356549,...,2.406951,1.886980,1.164061,-0.133447,2.254361,1.0,-0.083950,0.184332,NaN,4.358896
5724,2,0.069575,-0.87488,0.004105,-0.172768,-0.253396,-0.48648,-0.819258,-0.772763,0.609824,...,1.688536,1.129351,1.041293,0.597033,1.431506,1.0,0.070123,0.212544,NaN,3.211818
5725,2,0.069575,-0.87488,0.004105,-0.172768,-0.253396,-0.48648,-0.819258,-0.772763,0.356549,...,1.202035,0.640703,0.817736,1.265748,0.058954,1.0,0.301813,0.469022,NaN,2.294156
5726,2,0.069575,-0.87488,0.004105,-0.172768,-0.253396,-0.48648,-0.819258,-0.752127,-0.209366,...,1.192612,0.571852,0.988417,0.858098,0.290394,NaN,0.167998,0.350433,NaN,2.064740
5727,2,0.069575,-0.87488,0.004105,-0.172768,-0.253396,-0.48648,-0.819258,-0.772763,-0.909831,...,1.312224,0.681567,0.932523,1.157063,0.077092,1.0,0.070123,0.298037,NaN,1.835325
5728,2,0.069575,-0.87488,0.004105,-0.172768,-0.253396,-0.48648,-0.819258,-0.608694,-0.682594,...,1.311452,0.667614,1.120287,0.615888,0.403471,NaN,-0.083457,0.029404,NaN,1.605909
5729,2,0.069575,-0.87488,0.004105,-0.172768,-0.253396,-0.48648,-0.819258,-0.416885,-0.656555,...,1.074359,0.699543,0.813220,0.117173,1.243536,2.0,-0.277410,-0.300412,NaN,1.376494
5730,2,0.069575,-0.87488,0.004105,-0.172768,-0.253396,-0.48648,-0.819258,-0.539016,-0.486514,...,1.312254,0.667059,1.141220,0.533161,0.404521,NaN,-0.190362,-0.087210,NaN,1.147078
5731,2,0.069575,-0.87488,0.004105,-0.172768,-0.253396,-0.48648,-0.819258,-0.594824,-0.403279,...,1.294351,0.758900,0.863633,0.574730,0.486808,1.0,-0.161566,0.041559,NaN,0.917662


In [59]:
(4*(4+1))/2

10.0

In [57]:
len(in_df[ref_df.RID==3])

5

In [40]:
gind = ref_df[ref_df.RID==rid].index
x_tbl = in_df.loc[gind].sort_values('Month_bl',ascending=True)
y_tbl = ref_df.loc[x_tbl.index]
N_timepoints = len(x_tbl) - 1
N_rows = round((N_timepoints * (N_timepoints+1)/2))
ycols = [x for x in y_tbl.columns if 'y_' in x]
cols = ['RID'] + x_tbl.columns.tolist() + ycols + ['delta_t']
final_output = pandas.DataFrame(index=range(N_rows), columns = cols)

time_vec = x_tbl['Month'].values
cntr = 0
cumul = 0
while cntr < N_timepoints:
    jnk = pandas.concat([x_tbl.loc[x_tbl.index[cntr]],
                         y_tbl.loc[y_tbl.index[cntr]][['RID']+ycols]])
    row = jnk[cols[:-1]]
    iTime = row['Month']
    dTs = [x - iTime for x in time_vec[(cntr+1):]]
    for i in range(len(dTs)):
        final_output.loc[final_output.index[cumul+i],:] = row
        final_output.loc[final_output.index[cumul+i],'delta_t'] = dTs[i]
    cumul += len(dTs)
    cntr += 1
final_output

,RID,AGE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,PTMARRY,APOE4,CDRSB,MMSE,...,ST98TS_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST99CV_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST99SA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST99TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST99TS_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST9SV_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,y_DX,y_ADAS11,y_ADAS13,delta_t
0,2,0.0695753,-0.87488,0.00410519,-0.172768,-0.253396,-0.48648,-0.819258,-0.772763,0.356549,...,-0.0231826,3.17636,2.62717,1.42111,-0.606512,3.28765,1,-0.0839499,0.0988391,0.229416
1,2,0.0695753,-0.87488,0.00410519,-0.172768,-0.253396,-0.48648,-0.819258,-0.772763,0.356549,...,-0.0231826,3.17636,2.62717,1.42111,-0.606512,3.28765,1,-0.0839499,0.0988391,1.37649
2,2,0.0695753,-0.87488,0.00410519,-0.172768,-0.253396,-0.48648,-0.819258,-0.772763,0.356549,...,-0.0231826,3.17636,2.62717,1.42111,-0.606512,3.28765,1,-0.0839499,0.0988391,2.29416
3,2,0.0695753,-0.87488,0.00410519,-0.172768,-0.253396,-0.48648,-0.819258,-0.772763,0.356549,...,-0.0231826,3.17636,2.62717,1.42111,-0.606512,3.28765,1,-0.0839499,0.0988391,2.52357
4,2,0.0695753,-0.87488,0.00410519,-0.172768,-0.253396,-0.48648,-0.819258,-0.772763,0.356549,...,-0.0231826,3.17636,2.62717,1.42111,-0.606512,3.28765,1,-0.0839499,0.0988391,2.75299
5,2,0.0695753,-0.87488,0.00410519,-0.172768,-0.253396,-0.48648,-0.819258,-0.772763,0.356549,...,-0.0231826,3.17636,2.62717,1.42111,-0.606512,3.28765,1,-0.0839499,0.0988391,2.9824
6,2,0.0695753,-0.87488,0.00410519,-0.172768,-0.253396,-0.48648,-0.819258,-0.772763,0.356549,...,-0.0231826,3.17636,2.62717,1.42111,-0.606512,3.28765,1,-0.0839499,0.0988391,3.21182
7,2,0.0695753,-0.87488,0.00410519,-0.172768,-0.253396,-0.48648,-0.819258,-0.772763,0.356549,...,-0.0231826,3.17636,2.62717,1.42111,-0.606512,3.28765,1,-0.0839499,0.0988391,3.44123
8,2,0.0695753,-0.87488,0.00410519,-0.172768,-0.253396,-0.48648,-0.819258,-0.772763,0.356549,...,-0.0231826,3.17636,2.62717,1.42111,-0.606512,3.28765,1,-0.0839499,0.0988391,3.67065
9,2,0.0695753,-0.87488,0.00410519,-0.172768,-0.253396,-0.48648,-0.819258,-0.772763,0.356549,...,-0.0231826,3.17636,2.62717,1.42111,-0.606512,3.28765,1,-0.0839499,0.0988391,3.90007


In [37]:
gind = ref_df[ref_df.RID==rid].index
x_tbl = in_df.loc[gind].sort_values('Month_bl',ascending=True)
y_tbl = ref_df.loc[x_tbl.index]
N_timepoints = len(x_tbl) - 1
N_rows = round((N_timepoints * (N_timepoints+1)/2))
ycols = [x for x in y_tbl.columns if 'y_' in x]
cols = ['RID'] + x_tbl.columns.tolist() + ycols + ['delta_t']
cntr=0
cumul = 0
final_output = pandas.DataFrame(index=range(N_rows), columns = cols)
time_vec = x_tbl['Month'].values
jnk = pandas.concat([x_tbl.loc[x_tbl.index[cntr]],
                             y_tbl.loc[y_tbl.index[cntr]][['RID']+ycols]])
row = jnk[cols[:-1]]
iTime = row['Month']
dTs = [x - iTime for x in time_vec[(cntr+1):]]
for i in range(len(dTs)):
    final_output.loc[final_output.index[cumul+i],:] = row
final_output.head()

,RID,AGE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,PTMARRY,APOE4,CDRSB,MMSE,...,ST98TS_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST99CV_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST99SA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST99TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST99TS_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST9SV_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,y_DX,y_ADAS11,y_ADAS13,delta_t
0,2,0.0695753,-0.87488,0.00410519,-0.172768,-0.253396,-0.48648,-0.819258,-0.772763,0.356549,...,-0.0231826,3.17636,2.62717,1.42111,-0.606512,3.28765,1,-0.0839499,0.0988391,NaN
1,2,0.0695753,-0.87488,0.00410519,-0.172768,-0.253396,-0.48648,-0.819258,-0.772763,0.356549,...,-0.0231826,3.17636,2.62717,1.42111,-0.606512,3.28765,1,-0.0839499,0.0988391,NaN
2,2,0.0695753,-0.87488,0.00410519,-0.172768,-0.253396,-0.48648,-0.819258,-0.772763,0.356549,...,-0.0231826,3.17636,2.62717,1.42111,-0.606512,3.28765,1,-0.0839499,0.0988391,NaN
3,2,0.0695753,-0.87488,0.00410519,-0.172768,-0.253396,-0.48648,-0.819258,-0.772763,0.356549,...,-0.0231826,3.17636,2.62717,1.42111,-0.606512,3.28765,1,-0.0839499,0.0988391,NaN
4,2,0.0695753,-0.87488,0.00410519,-0.172768,-0.253396,-0.48648,-0.819258,-0.772763,0.356549,...,-0.0231826,3.17636,2.62717,1.42111,-0.606512,3.28765,1,-0.0839499,0.0988391,NaN


In [34]:
dTs

[0.22941559617268381,
 1.3764935770361024,
 2.2941559617268377,
 2.5235715578995213,
 2.7529871540722053,
 2.9824027502448889,
 3.211818346417572,
 3.4412339425902561,
 3.6706495387629392,
 3.9000651349356232,
 4.1294807311083073,
 4.5883119234536753]

In [23]:
x_tbl['Month_bl']

0       -0.997464
5723    -0.773404
5724     0.359416
5725     1.305727
5726     1.509757
5727     1.758852
5728     1.990424
5729     2.225748
5730     2.451062
5731     2.661354
10836    2.897938
11571    3.160793
11959    3.591401
Name: Month_bl, dtype: float64

In [27]:
N_timepoints

13

In [28]:
(N_timepoints * (N_timepoints+1)/2)

91.0